# Use ChatGPT to classify transaction data

Use this file for testing the API and prompt tuning

Before you start, make sure you have an API key setup:
+ Go to https://platform.openai.com/api-keys
+ Generate key
+ Open PowerShell
+ Run: setx OPENAI_API_KEY "the-key-you-just-generated"

In [17]:
import pandas as pd, os, zipfile, json, tiktoken, statistics
from openai import OpenAI
from pydantic import BaseModel
from datetime import datetime

model = "gpt-4o"  # More expensive, older and doesn't support structured outputs or batching
model = "o4-mini" # Use this - it is slightly slower, but has more recent training data and supports structured outputs

# Output class component
class ReturnComponent(BaseModel):
    index: int
    brand: str
    code: float
    confidence: float
    note: str

# Output class
class ClassificationReturn(BaseModel):
    values: list[ReturnComponent]

In [3]:
try: # Check we have the API key installed
    apiKey = os.getenv("OPENAI_API_KEY")
    print(f'Found API key with length {len(apiKey)}')
except:
    print('Cannot find OPENAI_API_KEY environment variable')

Found API key with length 164


# Test to see we can call the OpenAI API

In [4]:
cont= """
Hi, could you provide a list of the first five primes, triangular numbers, squares and cubes please.
Format your response as a table that I can easily read into a pandas dataframe
"""

In [5]:
client = OpenAI()

response = client.chat.completions.create(
    model=model, 
    messages=[
        {"role": "user", "content": cont}
    ]
)
print(response.choices[0].message.content)

Here’s a simple CSV‐style table you can feed directly into pandas (e.g. via pd.read_csv or pd.read_table with sep=","):

prime,triangular,square,cube  
2,1,1,1  
3,3,4,8  
5,6,9,27  
7,10,16,64  
11,15,25,125  

Example in pandas:

```
import pandas as pd
from io import StringIO

data = """prime,triangular,square,cube
2,1,1,1
3,3,4,8
5,6,9,27
7,10,16,64
11,15,25,125
"""

df = pd.read_csv(StringIO(data))
print(df)
```


# Get inputs to API

In [ ]:
# Get the top 20 thousand brands
df = pd.read_parquet('top_20K.parquet')

# System message
system_message_content =  open("system_message.txt", "r", encoding="utf-8").read() # This reads : 

You are a transaction–brand classifier for UK bank data. Use the following bespoke COICOP-derived scheme:

Codes:
+ -1.00: Income – Wages, salaries, government benefits, charitable receipts (excludes dissaving/investment returns).
+ + -1.10: Wages and salaries – Income from labour.
+ + -1.20: Entitlements/benefits – Government benefit payments.
+ + -1.30: Charitable payments – Charitable receipts.

+ 0.00: Unclassified – Unidentifiable or net-zero transactions.
+ + 0.10: Internal transfers – Between same individual’s accounts; exclude purpose-specific transfers (e.g. savings).
+ + 0.20: No brand or unrecognised brands – Inc. ATM, payment providers and alphanumeric codes; truly multi-category brands → 13.10.

+ 1.00: Food & non-alcoholic beverages. (inc. supermarkets)
+ 2.00: Alcoholic beverages, tobacco & narcotics. (including off-licences)
+ 3.00: Clothing & footwear.
+ 4.00: Housing, water, electricity, gas & other fuels (incl. rent/mortgage).
+ 5.00: Furnishings, household equipment & maintenance.
+ 6.00: Health.
+ 7.00: Transport (incl. car-loan repayments and postal costs).
+ 8.00: Information & communication.
+ 9.00: Recreation, sport & culture (gambling extracted).
+ 10.00: Education services (incl. tuition & student-loan repayments).
+ 11.00: Restaurants & accommodation services.

+ 12.00: Insurance & financial services.
+ + 12.10: Insurance – Premiums & claim payouts.
+ + 12.20: Financial services general. (Inc. non-credit card account charges and overdraft interest) 
+ + 12.21: Credit cards – Fees & interest only; purchases go to merchant category.
+ + 12.22: Savings – Transfers into savings. (including no/unrecognised bands that indicate an intention to save)
+ + 12.23: Investments – Transactions with investment providers.
+ + 12.24: Pensions – Pension contributions & payouts.
+ + 12.25: Loans – Loan receipts & repayments; **if any other purpose is implied** (car, mortgage, home improvement, business, etc.), classify to that purpose’s code instead.

+ 13.00: Personal care, social protection & misc. goods & services.
+ + 13.10: General purpose retailers – Truly multi-category (e.g. Amazon).
+ + 13.20: Taxes – Payments & rebates. (Unless tax for a specific purpose, such as property taxes → 4.0, or vehicle taxes → 7.0)
+ + 13.30: Political donations and charitable giving

+ 14.00: Gambling (except lotteries)
+ 15.00: Lotteries

**COICOP overlap**

Codes 1 to 11 align with COICOP 2018: https://unstats.un.org/unsd/classifications/unsdclassifications/COICOP_2018_-_pre-edited_white_cover_version_-_2018-12-26.pdf
Reference this to resolve difficulties. Exception: gambling moved from 9 to 14 and 15.

**Dominant-product rule:**  

By default, assign each brand to its primary/dominant category. If a specific product or purpose is explicitly mentioned in the _brand_ (e.g. “car loan”), classify to that product’s code rather than the brand’s default.

**Prioritising input information**

The descriptions are there as supporting information. If it is clear from the brand alone how to classify, do so. If this cannot be done, then make a judgement based on the descriptions. Weight each description equally, but the brand name highest.

**Limiting output information**

There is an output field, 'note'. This should almost always be blank. Only provide information about difficult judgement calls.

**Classification clarifications**

These results will be joined to millions of transactions on the 'brand'. Therefore, you should weight what you know about the brand much more highly than what you see in the descriptions. 
+ If the brand is best known for credit cards, always classify to 12.21 (such as Barclaycard, AMEX and others)
+ Stationary, greetings cards, news and book stores go to 9
+ Large banks such as Barclays and Halifax → 12.20, if the brand contains no information about the financial product used

In [ ]:
client = OpenAI()

def get_chatgpt_return(client, model, system_message_content, in_df):
    records_json = dataframe_to_api_records(in_df)

    response = client.responses.parse(
        model=model,
        input=[
            {"role":"system", "content": system_message_content},
            {
                "role": "user",
                "content": records_json,
            },
        ],
        text_format=ClassificationReturn,
    )
    event = response.output_parsed
    out_df = pd.DataFrame([comp.model_dump() for comp in event.values]).set_index('index', drop=True)   
    return response, out_df

def dataframe_to_api_records(df: pd.DataFrame) -> str:
    """
    Convert a DataFrame into the JSON string for the `records` argument
    of the classify_brands function.
    """
    records = []
    # Iterate over rows in the DataFrame
    for idx, row in df.iterrows():
        # Collect up to three description fields (adjust if you have more/less)
        descriptions = [
            row.get('description1'),
            row.get('description2'),
            row.get('description3')
        ]
        # Filter out any missing or NaN descriptions
        descriptions = [desc for desc in descriptions if pd.notna(desc)]
        
        record = {
            "index": int(idx),
            "brand": row['brand'],
            "descriptions": descriptions
        }
        records.append(record)
    
    # Convert to a JSON-formatted string
    return json.dumps({"records": records}, ensure_ascii=False)

In [ ]:
system_message_content =  open("system_message.txt", "r", encoding="utf-8").read() # Get latest version of system prompt

# Try a few random transactions
in_df = df.iloc[:1000].sample(10)
response, out_df = get_chatgpt_return(client, model, system_message_content, in_df)
in_out_df = pd.merge(out_df, in_df, left_index=True, right_index=True)
in_out_df.to_excel('test_output.xlsx')

In [ ]:
# Vary number of records to work out how many records we use for each call
inc = 0
rdf = []
responses = []
for i in range(10, 101, 10):
    imin = inc
    imax = inc + i
    inc = imax
    in_df = df.iloc[imin:imax]

    response, out_df = get_chatgpt_return(client, model, system_message_content, in_df)
    
    intok = response.usage.input_tokens
    outtok = response.usage.output_tokens

    print(f'ChatGPT returned for {i} records\t({intok},{outtok},{intok+outtok}) tokens\t {imin},\t {in_df.iloc[0,0]},\t through {imax},\t {in_df.iloc[-1,0]}')

    rdf += [pd.merge(out_df, in_df, left_index=True, right_index=True)]
    pd.concat(rdf).to_excel('test_output.xlsx')
    responses += [response]

ChatGPT returned for 10 records	(1601,1745,3346) tokens	 0,	 amazon,	 through 10,	 transport london
ChatGPT returned for 20 records	(2092,3274,5366) tokens	 10,	 morrisons,	 through 30,	 eat
ChatGPT returned for 30 records	(2604,3472,6076) tokens	 30,	 revolut,	 through 60,	 amex
ChatGPT returned for 40 records	(3062,3391,6453) tokens	 60,	 netflix,	 through 100,	 mobilechannel ft
ChatGPT returned for 50 records	(3732,5649,9381) tokens	 100,	 argos,	 through 150,	 giffgaff
ChatGPT returned for 60 records	(4061,5312,9373) tokens	 150,	 ikea,	 through 210,	 utilita energy
ChatGPT returned for 70 records	(4773,7119,11892) tokens	 210,	 nandos,	 through 280,	 currys
ChatGPT returned for 80 records	(5380,6949,12329) tokens	 280,	 account fee,	 through 360,	 burton
ChatGPT returned for 90 records	(5845,7453,13298) tokens	 360,	 pbz,	 through 450,	 national health service
ChatGPT returned for 100 records	(6173,8689,14862) tokens	 450,	 centra,	 through 550,	 kings arms


In [ ]:
# Does the return match the input?
rdf = pd.concat(rdf)
print(sum(rdf.brand_x != rdf.brand_y), 'misalignments')

0 misalignments


# Estimate max tokens

In [16]:
# response_tokens = enc.encode(response)
for response in responses:
    intok = response.usage.input_tokens
    outtok = response.usage.output_tokens
    print(intok, outtok, intok+outtok)

1316 2078 3394
1807 3934 5741
2319 4063 6382
2777 6167 8944
3447 4811 8258
3776 5816 9592
4488 5219 9707
5095 7744 12839
5560 7912 13472
5888 9694 15582
